In [1]:
import time
from cmath import *
from matplotlib.colors import hsv_to_rgb
import cv2
from tqdm import tqdm

In [32]:
def create_ppm(dim, pixels, filename):
    with open(filename, 'w') as ofile:
        header = f"""P3\n{dim[0]} {dim[1]}\n{max_pixel_val}\n"""
        ofile.write(header)
        for pixel in pixels:
            pixel_repr = " ".join([str(x) for x  in pixel]) + "\n"
            ofile.write(pixel_repr)


def create_pixels(dim, function, max_pixel_val = 255, zoom = 1, center = (0,0)):
    pixels = []
    width, height = dim
    i = 0
    for v in range(height):
        # Transform camera
        y = v - height/2 - center[1] 
        # y is the distance from the top because of how pixel arrays work
        y = y/zoom
        for u in range(width):
            i += 1
            # Transform camera
            x = u - width/2 + center[0]
            x = x/zoom
            
            pixel = function(x, y)
            pixel = tuple(int(pixel_val) % (max_pixel_val+1) for pixel_val in pixel)
            if (i) < 10:
                print(x, y, pixel)
            pixels.append(pixel)
    return pixels

In [67]:
def mandelbrot(x, y):
    #u, v = 
    iterations = 100
    c = complex(x,y)
    zn = 0
    for i in range(iterations):
        new_zn = zn*zn + c
        if new_zn == zn:
            # Converges
            return (0, 0, 0)
        try:
            if abs(new_zn) <= abs(zn) and i > 20:
                # Converges
                return (0, 0, 0)
        except OverflowError:
            # Diverges
            return tuple(int(x) for x in hsv_to_rgb((abs(1 - i/iterations)/1.5,1,1))*255)
        if ("nan" in str(new_zn)) or ("inf" in str(new_zn)):
            return tuple(int(x) for x in hsv_to_rgb((abs(1 - i/iterations)/1.5,1,1))*255)
        #if abs(new_zn.real) > x*100 or abs(new_zn.imag) > x*100:
            # Diverges
            #return tuple(int(x) for x in hsv_to_rgb((abs(1 - i/iterations),1,1))*255)
            #print(new_zn)
            #return tuple(int(x) for x in hsv_to_rgb((abs(i/100),1,1))*255)
    #return tuple(int(x) for x in hsv_to_rgb((abs(new_zn),1,1))*255)
        zn = new_zn
    return tuple(int(x) for x in hsv_to_rgb((abs(1 - i/iterations)/1.5,1,1))*255)


def sin_of_square(x, y):
    #u, v = 
    iterations = 100
    c = complex(x,y)
    zn = 0
    new_zn = zn
    #for i in tqdm(range(iterations)):
    for i in range(iterations):
        hue = 1 - abs(i/iterations)/2
        hue = (hue**0.7)/1.7
        rgb_color = tuple(int(x) for x in hsv_to_rgb((hue,1,1))*255)
        
        if ("nan" in str(new_zn)) or ("inf" in str(new_zn)):
            hue = 1 - abs(i/iterations)
            hue = (hue**0.7)/1.7
            return rgb_color
        try:
            new_zn = zn*sin(zn*zn) + c
        except (OverflowError, ValueError):
            # Diverges
            return rgb_color
        if new_zn == zn:
            # Converges
            return (0, 0, 0)
        try:
            if abs(new_zn) <= abs(zn) and i > iterations//3:
                # Converges
                return (0, 0, 0)
            if abs(new_zn) == 0:
                return (0,0,255)
        except OverflowError:
            # Diverges
            return rgb_color
        #if abs(new_zn.real) > x*100 or abs(new_zn.imag) > x*100:
            # Diverges
            #return tuple(int(x) for x in hsv_to_rgb((abs(1 - i/iterations),1,1))*255)
            #print(new_zn)
            #return tuple(int(x) for x in hsv_to_rgb((abs(i/100),1,1))*255)
    #return tuple(int(x) for x in hsv_to_rgb((abs(new_zn),1,1))*255)
        zn = new_zn
    return rgb_color


def fractal(x, y):
    #u, v = 
    iterations = 100
    c = complex(x,y)
    zn = 0
    new_zn = zn
    #for i in tqdm(range(iterations)):
    for i in range(iterations):
        # Make color stuff
        hue = 1 - abs(i/iterations)/2
        hue = (hue**0.7)/1.7
        rgb_color = tuple(int(x) for x in hsv_to_rgb((hue,1,1))*255)
        
        # Iteration stuff
        if ("nan" in str(new_zn)) or ("inf" in str(new_zn)):
            return rgb_color
        try:
            # The equation
            new_zn = zn*sin(zn + c) + c
        except (OverflowError, ValueError):
            # Diverges
            return rgb_color
        if new_zn == zn:
            # Converges
            return (0, 0, 0)
        try:
            if abs(new_zn) <= abs(zn) and i > iterations//3:
                # Converges
                return (0, 0, 0)
            if abs(new_zn) == 0:
                return (0,0,255)
        except OverflowError:
            # Diverges
            return rgb_color
        zn = new_zn
    return rgb_color


## Main
#Width, height
dim = (1920,1080)
max_pixel_val = 255
zoom = 200
center = (-500, 0)


start = time.time()
filename = "test" + str(int(time.time()))[-6:]
pixels = create_pixels(dim, fractal, zoom=zoom, center=center)
print(len(pixels))
create_ppm(dim, pixels, f"images/{filename}.ppm")
cv2.imwrite(f"images/{filename}.png", cv2.imread(f"images/{filename}.ppm")) #ppm to png
end = time.time()
print(f"duration: {end - start}s")

-7.3 -2.7 (0, 129, 255)
-7.295 -2.7 (0, 129, 255)
-7.29 -2.7 (0, 129, 255)
-7.285 -2.7 (0, 129, 255)
-7.28 -2.7 (0, 129, 255)
-7.275 -2.7 (0, 129, 255)
-7.27 -2.7 (0, 129, 255)
-7.265 -2.7 (0, 129, 255)
-7.26 -2.7 (0, 129, 255)
2073600
duration: 793.6701428890228s


In [ ]:
import os 
import cv2 
from glob import glob 

cwd = os.getcwd()
input_dir = os.path.join(cwd, "input\\*.ppm")    
ppms = glob(input_dir)   

counter = 1 

for ppm in ppms: 
    cv2.imwrite(str(counter)+".png", cv2.imread(ppm))
    counter += 1 